# AI hub 지능형 스마트팜 통합 데이터(토마토)

- https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100

### 1. 이미지 라벨링 데이터 json -> DataFrame

- tqdm 모듈은 시간 측정을 위해 추가했습니다.
- 두 함수는 json이 파이썬으로 로드 되었을 때 잘 작동합니다.
    - mk_tomato_df(), mk_growth_df()
- 소요시간, 메모리 고려해서 코드는 따로 작성했습니다.

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Flatten, Input, concatenate
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout

from tensorflow.keras.optimizers import SGD, Adam, RMSprop
import tensorflow.keras.utils as utils

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score

#
from datetime import datetime
import cv2
import zipfile
import shutil
import os
# import glob
from glob import glob
import json

# 소요 시간 확인
from tqdm import tqdm

In [128]:
# 경로 
tomato_dir = "C:/juna27/PythonWork/AI/data/tomato_label/지능형 스마트팜 통합데이터(토마토)_json/TL_a1생장길이/a1.생장길이"

tomato_json_data = []
for file_name in tqdm(os.listdir(tomato_dir)):
    if file_name.endswith(".json"):
        file_path = os.path.join(tomato_dir, file_name)
        with open(file_path, "r", encoding="utf-8") as target_file:
            tomato_json_data.append(json.load(target_file))

100%|██████████████████████████████████████████████████████████████████████████| 17500/17500 [00:04<00:00, 4310.85it/s]


In [129]:
json_data = tomato_json_data

In [131]:
# json_data[0]

In [62]:
def mk_tomato_df(json_data):
    tomato_list = []
    for i in tqdm(range(len(json_data))):
        data = {
        'imagePath': json_data[i]['imagePath'],
#         'imageData': json_data[i]['imageData'],
        'imageHeight': json_data[i]['imageHeight'],
        'imageWidth': json_data[i]['imageWidth'],
        'fileVersion': json_data[i]['file_attributes']['fileVersion'],
        'farmId': json_data[i]['file_attributes']['farmId'],
        'week': json_data[i]['file_attributes']['week'],
        'number': json_data[i]['file_attributes']['number'],
        'type': json_data[i]['file_attributes']['type'],
        'flowerCluster': json_data[i]['file_attributes']['flowerCluster'],
        'date': json_data[i]['file_attributes']['date']
        }
        tomato_list.append(data)
        df = pd.DataFrame(tomato_list)
    return df

In [63]:
mk_tomato_df(json_data).head()

100%|████████████████████████████████████████████████████████████████████████████| 17500/17500 [04:14<00:00, 68.73it/s]


,imagePath,imageHeight,imageWidth,fileVersion,farmId,week,number,type,flowerCluster,date
0,V001_tom1_39_001_a1_00_20210930_14_00134902_49...,1920,1080,V001,tom1,39,001,a1,00,20210930
1,V001_tom1_39_006_a1_00_20210930_14_00130645_49...,1920,1080,V001,tom1,39,006,a1,00,20210930
2,V001_tom1_39_010_a1_00_20210930_14_00124109_49...,1920,1080,V001,tom1,39,010,a1,00,20210930
3,V001_tom1_39_011_a1_00_20210930_14_00122602_49...,1920,1080,V001,tom1,39,011,a1,00,20210930
4,V001_tom1_39_011_a1_00_20210930_14_00124155_49...,1920,1080,V001,tom1,39,011,a1,00,20210930


In [132]:
tomato_a1_df = mk_tomato_df(json_data)

100%|████████████████████████████████████████████████████████████████████████████| 17500/17500 [04:13<00:00, 69.03it/s]


In [133]:
tomato_a1_df

,imagePath,imageHeight,imageWidth,fileVersion,farmId,week,number,type,flowerCluster,date
0,V001_tom1_39_001_a1_00_20210930_14_00134902_49...,1920,1080,V001,tom1,39,001,a1,00,20210930
1,V001_tom1_39_006_a1_00_20210930_14_00130645_49...,1920,1080,V001,tom1,39,006,a1,00,20210930
2,V001_tom1_39_010_a1_00_20210930_14_00124109_49...,1920,1080,V001,tom1,39,010,a1,00,20210930
3,V001_tom1_39_011_a1_00_20210930_14_00122602_49...,1920,1080,V001,tom1,39,011,a1,00,20210930
4,V001_tom1_39_011_a1_00_20210930_14_00124155_49...,1920,1080,V001,tom1,39,011,a1,00,20210930
...,...,...,...,...,...,...,...,...,...,...
17495,V001_tom6_47_097_a1_00_20211124_12_03094115_76...,1920,1080,V001,tom6,47,097,a1,00,20211124
17496,V001_tom6_47_097_a1_00_20211124_12_04094115_76...,1920,1080,V001,tom6,47,097,a1,00,20211124
17497,V001_tom6_47_098_a1_00_20211124_12_01095035_76...,1920,1080,V001,tom6,47,098,a1,00,20211124
17498,V001_tom6_47_098_a1_00_20211124_12_02095035_76...,1920,1080,V001,tom6,47,098,a1,00,20211124


In [31]:
tomato_a1_df.isnull().sum()

imagePath        0
imageHeight      0
imageWidth       0
fileVersion      0
farmId           0
week             0
number           0
type             0
flowerCluster    0
date             0
dtype: int64

In [8]:
def mk_growth_df(json_data):
    growth_list = []
    for i in tqdm(json_data):
        growth_indicators = i.get("growth_indicators", {})

        item = {}
        # growth_indicators 키, 값 반복 => item에 추가
        for key, value in growth_indicators.items():
            item[key] = value
            
        # imagePath 추가 / 임시 키값 선정
        item["imagePath"] = i["imagePath"]
        
        growth_list.append(item)

    df = pd.DataFrame(growth_list)
    return df

In [134]:
mk_growth_df(json_data)

100%|████████████████████████████████████████████████████████████████████████| 17500/17500 [00:00<00:00, 674343.99it/s]


,plantHeight,weeklyGrowth,imagePath
0,37.10,37.10,V001_tom1_39_001_a1_00_20210930_14_00134902_49...
1,35.15,35.15,V001_tom1_39_006_a1_00_20210930_14_00130645_49...
2,38.69,38.69,V001_tom1_39_010_a1_00_20210930_14_00124109_49...
3,43.58,43.58,V001_tom1_39_011_a1_00_20210930_14_00122602_49...
4,NaN,NaN,V001_tom1_39_011_a1_00_20210930_14_00124155_49...
...,...,...,...
17495,24.35,24.35,V001_tom6_47_097_a1_00_20211124_12_03094115_76...
17496,21.59,21.59,V001_tom6_47_097_a1_00_20211124_12_04094115_76...
17497,NaN,NaN,V001_tom6_47_098_a1_00_20211124_12_01095035_76...
17498,19.86,19.86,V001_tom6_47_098_a1_00_20211124_12_02095035_76...


In [135]:
growth_a1_df = mk_growth_df(json_data)

100%|███████████████████████████████████████████████████████████████████████| 17500/17500 [00:00<00:00, 1027842.94it/s]


In [136]:
growth_a1_df.isnull().sum()

plantHeight     3333
weeklyGrowth    3333
imagePath          0
dtype: int64

In [56]:
# a2 화방높이

In [9]:
tomato_dir = "C:/juna27/PythonWork/AI/data/tomato_label/지능형 스마트팜 통합데이터(토마토)_json/TL_a2화방높이/a2.화방높이"

tomato_json_data = []
for file_name in tqdm(os.listdir(tomato_dir)):
    if file_name.endswith(".json"):
        file_path = os.path.join(tomato_dir, file_name)
        with open(file_path, "r", encoding="utf-8") as target_file:
            tomato_json_data.append(json.load(target_file))

100%|████████████████████████████████████████████████████████████████████████████| 3500/3500 [00:00<00:00, 4874.12it/s]


In [10]:
a2_data = tomato_json_data

In [11]:
tomato_a2_df = mk_tomato_df(a2_data)

100%|█████████████████████████████████████████████████████████████████████████████| 3500/3500 [00:07<00:00, 455.70it/s]


In [12]:
tomato_a2_df

,imagePath,imageHeight,imageWidth,fileVersion,farmId,week,number,type,flowerCluster,date
0,V001_tom1_00_052_a2_18_20220101_27_00092524_40...,1920,1080,V001,tom1,00,052,a2,18,20220101
1,V001_tom1_00_055_a2_16_20220101_27_00093208_40...,1920,1080,V001,tom1,00,055,a2,16,20220101
2,V001_tom1_00_056_a2_16_20220101_27_00093417_40...,1920,1080,V001,tom1,00,056,a2,16,20220101
3,V001_tom1_00_057_a2_16_20220101_27_00093620_40...,1920,1080,V001,tom1,00,057,a2,16,20220101
4,V001_tom1_00_058_a2_17_20220101_27_00093815_40...,1920,1080,V001,tom1,00,058,a2,17,20220101
...,...,...,...,...,...,...,...,...,...,...
3495,V001_tom4_38_074_a2_04_20210925_07_01152423_87...,1920,1080,V001,tom4,38,074,a2,04,20210925
3496,V001_tom4_38_075_a2_03_20210919_06_01120253_67...,1920,1080,V001,tom4,38,075,a2,03,20210919
3497,V001_tom4_38_075_a2_04_20210925_07_00153038_87...,1920,1080,V001,tom4,38,075,a2,04,20210925
3498,V001_tom4_38_075_a2_04_20210925_07_01153038_87...,1920,1080,V001,tom4,38,075,a2,04,20210925


In [13]:
tomato_a2_df.isnull().sum()

imagePath        0
imageHeight      0
imageWidth       0
fileVersion      0
farmId           0
week             0
number           0
type             0
flowerCluster    0
date             0
dtype: int64

In [14]:
growth_a2_df = mk_growth_df(a2_data)

100%|██████████████████████████████████████████████████████████████████████████| 3500/3500 [00:00<00:00, 881738.48it/s]


In [15]:
growth_a2_df

,heightOfTheFlowerTruss,imagePath
0,27.50,V001_tom1_00_052_a2_18_20220101_27_00092524_40...
1,46.69,V001_tom1_00_055_a2_16_20220101_27_00093208_40...
2,49.63,V001_tom1_00_056_a2_16_20220101_27_00093417_40...
3,28.67,V001_tom1_00_057_a2_16_20220101_27_00093620_40...
4,36.50,V001_tom1_00_058_a2_17_20220101_27_00093815_40...
...,...,...
3495,17.91,V001_tom4_38_074_a2_04_20210925_07_01152423_87...
3496,NaN,V001_tom4_38_075_a2_03_20210919_06_01120253_67...
3497,NaN,V001_tom4_38_075_a2_04_20210925_07_00153038_87...
3498,20.11,V001_tom4_38_075_a2_04_20210925_07_01153038_87...


In [16]:
growth_a2_df.isnull().sum()

heightOfTheFlowerTruss    328
imagePath                   0
dtype: int64

In [63]:
# a3 줄기두께

In [73]:
tomato_dir = "C:/juna27/PythonWork/AI/data/tomato_label/지능형 스마트팜 통합데이터(토마토)_json/TL_a3줄기두께/a3.줄기두께"

tomato_json_data = []
for file_name in tqdm(os.listdir(tomato_dir)):
    if file_name.endswith(".json"):
        file_path = os.path.join(tomato_dir, file_name)
        with open(file_path, "r", encoding="utf-8") as target_file:
            tomato_json_data.append(json.load(target_file))

100%|████████████████████████████████████████████████████████████████████████████| 7001/7001 [00:05<00:00, 1205.37it/s]


In [74]:
a3_data = tomato_json_data

In [75]:
tomato_a3_df = mk_tomato_df(a3_data)
tomato_a3_df.isnull().sum()

100%|█████████████████████████████████████████████████████████████████████████████| 7001/7001 [00:30<00:00, 228.76it/s]


imagePath        0
imageHeight      0
imageWidth       0
fileVersion      0
farmId           0
week             0
number           0
type             0
flowerCluster    0
date             0
dtype: int64

In [78]:
tomato_a3_df

,imagePath,imageHeight,imageWidth,fileVersion,farmId,week,number,type,flowerCluster,date
0,V001_tom1_36_061_a3_00_20210909_01_00075303_40...,1920,1080,V001,tom1,36,061,a3,00,20210909
1,V001_tom1_36_065_a3_00_20210909_01_00072553_40...,1920,1080,V001,tom1,36,065,a3,00,20210909
2,V001_tom1_36_068_a3_00_20210909_01_00070626_40...,1920,1080,V001,tom1,36,068,a3,00,20210909
3,V001_tom1_36_070_a3_00_20210909_01_00063942_40...,1920,1080,V001,tom1,36,070,a3,00,20210909
4,V001_tom1_36_076_a3_00_20210908_01_00105429_38...,1920,1080,V001,tom1,36,076,a3,00,20210908
...,...,...,...,...,...,...,...,...,...,...
6996,V001_tom6_44_087_a3_00_20211102_09_00151259_93...,1920,1080,V001,tom6,44,087,a3,00,20211102
6997,V001_tom6_44_091_a3_00_20211103_09_01093615_76...,1920,1080,V001,tom6,44,091,a3,00,20211103
6998,V001_tom6_44_091_a3_00_20211103_09_02093615_76...,1920,1080,V001,tom6,44,091,a3,00,20211103
6999,V001_tom6_44_092_a3_00_20211103_09_01094401_76...,1920,1080,V001,tom6,44,092,a3,00,20211103


In [76]:
growth_a3_df = mk_growth_df(a3_data)
growth_a3_df.isnull().sum()

100%|██████████████████████████████████████████████████████████████████████████| 7001/7001 [00:00<00:00, 995164.62it/s]


stemDiameter    307
imagePath         0
dtype: int64

In [79]:
growth_a3_df

,stemDiameter,imagePath
0,0.69,V001_tom1_36_061_a3_00_20210909_01_00075303_40...
1,0.69,V001_tom1_36_065_a3_00_20210909_01_00072553_40...
2,0.85,V001_tom1_36_068_a3_00_20210909_01_00070626_40...
3,0.96,V001_tom1_36_070_a3_00_20210909_01_00063942_40...
4,0.99,V001_tom1_36_076_a3_00_20210908_01_00105429_38...
...,...,...
6996,0.74,V001_tom6_44_087_a3_00_20211102_09_00151259_93...
6997,1.45,V001_tom6_44_091_a3_00_20211103_09_01093615_76...
6998,1.54,V001_tom6_44_091_a3_00_20211103_09_02093615_76...
6999,1.27,V001_tom6_44_092_a3_00_20211103_09_01094401_76...


In [80]:
# a4 엽장엽폭

In [81]:
tomato_dir = "C:/juna27/PythonWork/AI/data/tomato_label/지능형 스마트팜 통합데이터(토마토)_json/TL_a4엽장엽폭/a4.엽장엽폭"

tomato_json_data = []
for file_name in tqdm(os.listdir(tomato_dir)):
    if file_name.endswith(".json"):
        file_path = os.path.join(tomato_dir, file_name)
        with open(file_path, "r", encoding="utf-8") as target_file:
            tomato_json_data.append(json.load(target_file))

100%|█████████████████████████████████████████████████████████████████████████████| 4899/4899 [00:05<00:00, 964.12it/s]


In [82]:
a4_data = tomato_json_data

In [83]:
tomato_a4_df = mk_tomato_df(a4_data)
tomato_a4_df.isnull().sum()

100%|█████████████████████████████████████████████████████████████████████████████| 4899/4899 [00:16<00:00, 304.28it/s]


imagePath        0
imageHeight      0
imageWidth       0
fileVersion      0
farmId           0
week             0
number           0
type             0
flowerCluster    0
date             0
dtype: int64

In [84]:
tomato_a4_df

,imagePath,imageHeight,imageWidth,fileVersion,farmId,week,number,type,flowerCluster,date
0,V001_tom1_00_051_a4_00_20220101_27_02092420_40...,1920,1080,V001,tom1,00,051,a4,00,20220101
1,V001_tom1_00_052_a4_00_20220101_27_01092454_40...,1920,1080,V001,tom1,00,052,a4,00,20220101
2,V001_tom1_00_052_a4_00_20220101_27_02092454_40...,1920,1080,V001,tom1,00,052,a4,00,20220101
3,V001_tom1_00_054_a4_00_20220101_27_01092735_40...,1920,1080,V001,tom1,00,054,a4,00,20220101
4,V001_tom1_00_056_a4_00_20220101_27_00093455_40...,1920,1080,V001,tom1,00,056,a4,00,20220101
...,...,...,...,...,...,...,...,...,...,...
4894,V001_tom6_47_007_a4_00_20211123_12_02141308_37...,1920,1080,V001,tom6,47,007,a4,00,20211123
4895,V001_tom6_47_008_a4_00_20211123_12_00141720_37...,1920,1080,V001,tom6,47,008,a4,00,20211123
4896,V001_tom6_47_008_a4_00_20211123_12_02141720_37...,1920,1080,V001,tom6,47,008,a4,00,20211123
4897,V001_tom6_47_009_a4_00_20211123_12_00142236_37...,1920,1080,V001,tom6,47,009,a4,00,20211123


In [85]:
growth_a4_df = mk_growth_df(a4_data)
growth_a4_df.isnull().sum()

100%|██████████████████████████████████████████████████████████████████████████| 4899/4899 [00:00<00:00, 307024.11it/s]


numberOfLeaves     179
imagePath            0
leafLength        4440
leafWidth         4440
dtype: int64

In [86]:
growth_a4_df

,numberOfLeaves,imagePath,leafLength,leafWidth
0,20,V001_tom1_00_051_a4_00_20220101_27_02092420_40...,NaN,NaN
1,20,V001_tom1_00_052_a4_00_20220101_27_01092454_40...,NaN,NaN
2,20,V001_tom1_00_052_a4_00_20220101_27_02092454_40...,NaN,NaN
3,19,V001_tom1_00_054_a4_00_20220101_27_01092735_40...,NaN,NaN
4,19,V001_tom1_00_056_a4_00_20220101_27_00093455_40...,NaN,NaN
...,...,...,...,...
4894,8,V001_tom6_47_007_a4_00_20211123_12_02141308_37...,NaN,NaN
4895,5,V001_tom6_47_008_a4_00_20211123_12_00141720_37...,NaN,NaN
4896,5,V001_tom6_47_008_a4_00_20211123_12_02141720_37...,NaN,NaN
4897,12,V001_tom6_47_009_a4_00_20211123_12_00142236_37...,NaN,NaN


In [88]:
# b 개화군

In [89]:
tomato_dir = "C:/juna27/PythonWork/AI/data/tomato_label/지능형 스마트팜 통합데이터(토마토)_json/TL_b개화군/b.개화군"

tomato_json_data = []
for file_name in tqdm(os.listdir(tomato_dir)):
    if file_name.endswith(".json"):
        file_path = os.path.join(tomato_dir, file_name)
        with open(file_path, "r", encoding="utf-8") as target_file:
            tomato_json_data.append(json.load(target_file))

100%|█████████████████████████████████████████████████████████████████████████████| 9100/9100 [00:20<00:00, 446.82it/s]


In [90]:
b1_data = tomato_json_data

In [91]:
tomato_b1_df = mk_tomato_df(b1_data)
tomato_b1_df.isnull().sum()

100%|█████████████████████████████████████████████████████████████████████████████| 9100/9100 [00:56<00:00, 160.71it/s]


imagePath        0
imageHeight      0
imageWidth       0
fileVersion      0
farmId           0
week             0
number           0
type             0
flowerCluster    0
date             0
dtype: int64

In [92]:
tomato_b1_df

,imagePath,imageHeight,imageWidth,fileVersion,farmId,week,number,type,flowerCluster,date
0,V001_tom1_00_063_b_16_20220101_27_00094916_401...,1920,1080,V001,tom1,00,063,b,16,20220101
1,V001_tom1_00_063_b_16_20220101_27_01094916_401...,1920,1080,V001,tom1,00,063,b,16,20220101
2,V001_tom1_00_063_b_16_20220101_27_02094916_401...,1920,1080,V001,tom1,00,063,b,16,20220101
3,V001_tom1_00_063_b_16_20220101_27_03094916_401...,1920,1080,V001,tom1,00,063,b,16,20220101
4,V001_tom1_00_063_b_16_20220101_27_04094916_401...,1920,1080,V001,tom1,00,063,b,16,20220101
...,...,...,...,...,...,...,...,...,...,...
9095,V001_tom6_46_072_b_06_20211115_11_00100004_766...,1920,1080,V001,tom6,46,072,b,06,20211115
9096,V001_tom6_46_072_b_06_20211115_11_02100004_766...,1920,1080,V001,tom6,46,072,b,06,20211115
9097,V001_tom6_46_072_b_06_20211115_11_03100004_766...,1920,1080,V001,tom6,46,072,b,06,20211115
9098,V001_tom6_46_072_b_06_20211115_11_04100004_766...,1920,1080,V001,tom6,46,072,b,06,20211115


In [93]:
growth_b1_df = mk_growth_df(b1_data)
growth_b1_df.isnull().sum()

100%|█████████████████████████████████████████████████████████████████████████| 9100/9100 [00:00<00:00, 1013547.36it/s]


floweringNode                0
numberOfTheFlower            0
numberOfTheFlowerPerTruss    0
imagePath                    0
dtype: int64

In [94]:
growth_b1_df

,floweringNode,numberOfTheFlower,numberOfTheFlowerPerTruss,imagePath
0,16,5,16,V001_tom1_00_063_b_16_20220101_27_00094916_401...
1,16,4,16,V001_tom1_00_063_b_16_20220101_27_01094916_401...
2,16,4,16,V001_tom1_00_063_b_16_20220101_27_02094916_401...
3,16,5,16,V001_tom1_00_063_b_16_20220101_27_03094916_401...
4,16,5,16,V001_tom1_00_063_b_16_20220101_27_04094916_401...
...,...,...,...,...
9095,06,2,06,V001_tom6_46_072_b_06_20211115_11_00100004_766...
9096,06,2,06,V001_tom6_46_072_b_06_20211115_11_02100004_766...
9097,06,2,06,V001_tom6_46_072_b_06_20211115_11_03100004_766...
9098,06,2,06,V001_tom6_46_072_b_06_20211115_11_04100004_766...


In [ ]:
# c 착과군

In [97]:
tomato_dir = "C:/juna27/PythonWork/AI/data/tomato_label/지능형 스마트팜 통합데이터(토마토)_json/TL_c착과군/c.착과군"

tomato_json_data = []
for file_name in tqdm(os.listdir(tomato_dir)):
    if file_name.endswith(".json"):
        file_path = os.path.join(tomato_dir, file_name)
        with open(file_path, "r", encoding="utf-8") as target_file:
            tomato_json_data.append(json.load(target_file))

100%|███████████████████████████████████████████████████████████████████████████| 66500/66500 [09:54<00:00, 111.90it/s]


In [98]:
c1_data = tomato_json_data

In [99]:
tomato_c1_df = mk_tomato_df(c1_data)
tomato_c1_df.isnull().sum()

100%|██████████████████████████████████████████████████████████████████████████| 66500/66500 [1:18:07<00:00, 14.19it/s]


imagePath        0
imageHeight      0
imageWidth       0
fileVersion      0
farmId           0
week             0
number           0
type             0
flowerCluster    0
date             0
dtype: int64

In [100]:
tomato_c1_df

,imagePath,imageHeight,imageWidth,fileVersion,farmId,week,number,type,flowerCluster,date
0,V001_tom1_00_061_c_14_20220101_27_00094434_401...,1920,1080,V001,tom1,00,061,c,14,20220101
1,V001_tom1_00_061_c_14_20220101_27_01094434_401...,1920,1080,V001,tom1,00,061,c,14,20220101
2,V001_tom1_00_061_c_14_20220101_27_03094434_401...,1920,1080,V001,tom1,00,061,c,14,20220101
3,V001_tom1_00_061_c_14_20220101_27_04094434_401...,1920,1080,V001,tom1,00,061,c,14,20220101
4,V001_tom1_00_061_c_15_20220101_27_00094401_401...,1920,1080,V001,tom1,00,061,c,15,20220101
...,...,...,...,...,...,...,...,...,...,...
66495,V001_tom6_48_013_c_02_20211130_13_00143955_376...,1920,1080,V001,tom6,48,013,c,02,20211130
66496,V001_tom6_48_013_c_02_20211130_13_02143955_376...,1920,1080,V001,tom6,48,013,c,02,20211130
66497,V001_tom6_48_013_c_02_20211130_13_03143955_376...,1920,1080,V001,tom6,48,013,c,02,20211130
66498,V001_tom6_48_013_c_02_20211130_13_04143955_376...,1920,1080,V001,tom6,48,013,c,02,20211130


In [101]:
growth_c1_df = mk_growth_df(c1_data)
growth_c1_df.isnull().sum()

100%|████████████████████████████████████████████████████████████████████████| 66500/66500 [00:00<00:00, 326291.96it/s]


numberOfFruitPerPlant        0
numberOfFruitPerTruss        0
fruitingNode                 0
imagePath                    0
numberOfTheFlower        58990
dtype: int64

In [102]:
growth_c1_df

,numberOfFruitPerPlant,numberOfFruitPerTruss,fruitingNode,imagePath,numberOfTheFlower
0,2,14,14,V001_tom1_00_061_c_14_20220101_27_00094434_401...,NaN
1,2,14,14,V001_tom1_00_061_c_14_20220101_27_01094434_401...,NaN
2,2,14,14,V001_tom1_00_061_c_14_20220101_27_03094434_401...,NaN
3,2,14,14,V001_tom1_00_061_c_14_20220101_27_04094434_401...,NaN
4,2,15,15,V001_tom1_00_061_c_15_20220101_27_00094401_401...,NaN
...,...,...,...,...,...
66495,5,02,02,V001_tom6_48_013_c_02_20211130_13_00143955_376...,NaN
66496,5,02,02,V001_tom6_48_013_c_02_20211130_13_02143955_376...,NaN
66497,5,02,02,V001_tom6_48_013_c_02_20211130_13_03143955_376...,NaN
66498,5,02,02,V001_tom6_48_013_c_02_20211130_13_04143955_376...,NaN


In [103]:
# d1 꽃_개화군

In [142]:
tomato_dir = "C:/juna27/PythonWork/AI/data/tomato_label/지능형 스마트팜 통합데이터(토마토)_json/TL_d1꽃_개화군/d1.꽃(개화군)"

tomato_json_data = []
for file_name in tqdm(os.listdir(tomato_dir)):
    if file_name.endswith(".json"):
        file_path = os.path.join(tomato_dir, file_name)
        with open(file_path, "r", encoding="utf-8") as target_file:
            tomato_json_data.append(json.load(target_file))

100%|████████████████████████████████████████████████████████████████████████████| 6999/6999 [00:03<00:00, 1756.14it/s]


In [143]:
d1_data = tomato_json_data

In [144]:
tomato_d1_df = mk_tomato_df(d1_data)
tomato_d1_df.isnull().sum()

100%|█████████████████████████████████████████████████████████████████████████████| 6999/6999 [00:34<00:00, 200.67it/s]


imagePath        0
imageHeight      0
imageWidth       0
fileVersion      0
farmId           0
week             0
number           0
type             0
flowerCluster    0
date             0
dtype: int64

In [145]:
tomato_d1_df

,imagePath,imageHeight,imageWidth,fileVersion,farmId,week,number,type,flowerCluster,date
0,V001_tom1_00_063_d1_16_20220101_27_06094916_40...,1920,1080,V001,tom1,00,063,d1,16,20220101
1,V001_tom1_00_063_d1_16_20220101_27_07094916_40...,1920,1080,V001,tom1,00,063,d1,16,20220101
2,V001_tom1_00_065_d1_17_20220101_27_05095632_40...,1920,1080,V001,tom1,00,065,d1,17,20220101
3,V001_tom1_00_065_d1_17_20220101_27_06095632_40...,1920,1080,V001,tom1,00,065,d1,17,20220101
4,V001_tom1_00_065_d1_17_20220101_27_07095632_40...,1920,1080,V001,tom1,00,065,d1,17,20220101
...,...,...,...,...,...,...,...,...,...,...
6994,V001_tom6_43_068_d1_0005_20211029_08_06100623_...,1920,1080,V001,tom6,43,068,d1,0005,20211029
6995,V001_tom6_43_069_d1_0004_20211029_08_05101220_...,1920,1080,V001,tom6,43,069,d1,0004,20211029
6996,V001_tom6_43_069_d1_0004_20211029_08_06101220_...,1920,1080,V001,tom6,43,069,d1,0004,20211029
6997,V001_tom6_43_069_d1_0004_20211029_08_07101220_...,1920,1080,V001,tom6,43,069,d1,0004,20211029


In [146]:
growth_d1_df = mk_growth_df(d1_data)
growth_d1_df.isnull().sum()

100%|██████████████████████████████████████████████████████████████████████████| 6999/6999 [00:00<00:00, 872468.09it/s]


floweringNode                         0
numberOfTheFlower                     0
numberOfTheFlowerPerTruss             0
imagePath                             0
numberOfTheFlowerFullyDeveloped    5027
dtype: int64

In [147]:
growth_d1_df

,floweringNode,numberOfTheFlower,numberOfTheFlowerPerTruss,imagePath,numberOfTheFlowerFullyDeveloped
0,16,1,16,V001_tom1_00_063_d1_16_20220101_27_06094916_40...,NaN
1,16,1,16,V001_tom1_00_063_d1_16_20220101_27_07094916_40...,NaN
2,17,1,17,V001_tom1_00_065_d1_17_20220101_27_05095632_40...,1.0
3,17,1,17,V001_tom1_00_065_d1_17_20220101_27_06095632_40...,NaN
4,17,1,17,V001_tom1_00_065_d1_17_20220101_27_07095632_40...,NaN
...,...,...,...,...,...
6994,0005,1,0005,V001_tom6_43_068_d1_0005_20211029_08_06100623_...,NaN
6995,0004,1,0004,V001_tom6_43_069_d1_0004_20211029_08_05101220_...,NaN
6996,0004,1,0004,V001_tom6_43_069_d1_0004_20211029_08_06101220_...,NaN
6997,0004,1,0004,V001_tom6_43_069_d1_0004_20211029_08_07101220_...,NaN


In [106]:
# d2 꽃_착과군

In [108]:
tomato_dir = "C:/juna27/PythonWork/AI/data/tomato_label/지능형 스마트팜 통합데이터(토마토)_json/TL_d2꽃_착과군/d2.꽃(착과군)"

tomato_json_data = []
for file_name in tqdm(os.listdir(tomato_dir)):
    if file_name.endswith(".json"):
        file_path = os.path.join(tomato_dir, file_name)
        with open(file_path, "r", encoding="utf-8") as target_file:
            tomato_json_data.append(json.load(target_file))

100%|█████████████████████████████████████████████████████████████████████████████| 2101/2101 [00:07<00:00, 288.43it/s]


In [109]:
d2_data = tomato_json_data

In [110]:
tomato_d2_df = mk_tomato_df(d2_data)
tomato_d2_df.isnull().sum()

100%|█████████████████████████████████████████████████████████████████████████████| 2101/2101 [00:03<00:00, 655.39it/s]


imagePath        0
imageHeight      0
imageWidth       0
fileVersion      0
farmId           0
week             0
number           0
type             0
flowerCluster    0
date             0
dtype: int64

In [111]:
tomato_d2_df

,imagePath,imageHeight,imageWidth,fileVersion,farmId,week,number,type,flowerCluster,date
0,V001_tom1_00_088_d2_13_20220102_27_07103453_38...,1920,1080,V001,tom1,00,088,d2,13,20220102
1,V001_tom1_36_064_d2_05_20210909_01_01072933_40...,1920,1080,V001,tom1,36,064,d2,05,20210909
2,V001_tom1_36_066_d2_05_20210909_01_01071725_40...,1920,1080,V001,tom1,36,066,d2,05,20210909
3,V001_tom1_36_067_d2_06_20210909_01_01071342_40...,1920,1080,V001,tom1,36,067,d2,06,20210909
4,V001_tom1_36_067_d2_06_20210909_01_02071342_40...,1920,1080,V001,tom1,36,067,d2,06,20210909
...,...,...,...,...,...,...,...,...,...,...
2096,V001_tom6_45_028_d2_05_20211111_10_08145027_37...,1920,1080,V001,tom6,45,028,d2,05,20211111
2097,V001_tom6_45_032_d2_06_20211111_10_06154228_37...,1920,1080,V001,tom6,45,032,d2,06,20211111
2098,V001_tom6_45_033_d2_05_20211112_10_06094611_37...,1920,1080,V001,tom6,45,033,d2,05,20211112
2099,V001_tom6_45_036_d2_06_20211108_10_04093423_93...,1920,1080,V001,tom6,45,036,d2,06,20211108


In [112]:
growth_d2_df = mk_growth_df(d2_data)
growth_d2_df.isnull().sum()

100%|█████████████████████████████████████████████████████████████████████████| 2101/2101 [00:00<00:00, 1064665.06it/s]


floweringNode                         0
numberOfTheFlower                     0
numberOfTheFlowerPerTruss             0
imagePath                             0
numberOfTheFlowerFullyDeveloped    1326
dtype: int64

In [113]:
growth_d2_df

,floweringNode,numberOfTheFlower,numberOfTheFlowerPerTruss,imagePath,numberOfTheFlowerFullyDeveloped
0,13,1,13,V001_tom1_00_088_d2_13_20220102_27_07103453_38...,NaN
1,05,1,05,V001_tom1_36_064_d2_05_20210909_01_01072933_40...,NaN
2,05,1,05,V001_tom1_36_066_d2_05_20210909_01_01071725_40...,NaN
3,06,1,06,V001_tom1_36_067_d2_06_20210909_01_01071342_40...,NaN
4,06,1,06,V001_tom1_36_067_d2_06_20210909_01_02071342_40...,NaN
...,...,...,...,...,...
2096,05,1,05,V001_tom6_45_028_d2_05_20211111_10_08145027_37...,NaN
2097,06,1,06,V001_tom6_45_032_d2_06_20211111_10_06154228_37...,1.0
2098,05,1,05,V001_tom6_45_033_d2_05_20211112_10_06094611_37...,NaN
2099,06,1,06,V001_tom6_45_036_d2_06_20211108_10_04093423_93...,NaN


In [114]:
# e 열매

In [115]:
tomato_dir = "C:/juna27/PythonWork/AI/data/tomato_label/지능형 스마트팜 통합데이터(토마토)_json/TL_e열매/e.열매"

tomato_json_data = []
for file_name in tqdm(os.listdir(tomato_dir)):
    if file_name.endswith(".json"):
        file_path = os.path.join(tomato_dir, file_name)
        with open(file_path, "r", encoding="utf-8") as target_file:
            tomato_json_data.append(json.load(target_file))

100%|████████████████████████████████████████████████████████████████████████████| 27300/27300 [05:43<00:00, 79.46it/s]


In [116]:
e1_data = tomato_json_data

In [117]:
tomato_e1_df = mk_tomato_df(e1_data)
tomato_e1_df.isnull().sum()

100%|████████████████████████████████████████████████████████████████████████████| 27300/27300 [11:57<00:00, 38.03it/s]


imagePath        0
imageHeight      0
imageWidth       0
fileVersion      0
farmId           0
week             0
number           0
type             0
flowerCluster    0
date             0
dtype: int64

In [118]:
tomato_e1_df

,imagePath,imageHeight,imageWidth,fileVersion,farmId,week,number,type,flowerCluster,date
0,V001_tom1_00_061_e_14_20220101_27_05094434_401...,1920,1080,V001,tom1,00,061,e,14,20220101
1,V001_tom1_00_061_e_15_20220101_27_06094401_401...,1920,1080,V001,tom1,00,061,e,15,20220101
2,V001_tom1_00_062_e_14_20220101_27_05094518_401...,1920,1080,V001,tom1,00,062,e,14,20220101
3,V001_tom1_00_062_e_15_20220101_27_05094550_401...,1920,1080,V001,tom1,00,062,e,15,20220101
4,V001_tom1_00_062_e_15_20220101_27_06094550_401...,1920,1080,V001,tom1,00,062,e,15,20220101
...,...,...,...,...,...,...,...,...,...,...
27295,V001_tom6_49_031_e_06_20211209_14_06104715_376...,1920,1080,V001,tom6,49,031,e,06,20211209
27296,V001_tom6_49_031_e_07_20211209_14_08104831_376...,1920,1080,V001,tom6,49,031,e,07,20211209
27297,V001_tom6_49_032_e_03_20211209_14_04155115_376...,1920,1080,V001,tom6,49,032,e,03,20211209
27298,V001_tom6_49_032_e_05_20211209_14_04150227_376...,1920,1080,V001,tom6,49,032,e,05,20211209


In [119]:
growth_e1_df = mk_growth_df(e1_data)
growth_e1_df.isnull().sum()

100%|████████████████████████████████████████████████████████████████████████| 27300/27300 [00:00<00:00, 744101.03it/s]


numberOfFruitPerPlant    0
numberOfFruitPerTruss    0
fruitingNode             0
imagePath                0
dtype: int64

In [120]:
growth_e1_df

,numberOfFruitPerPlant,numberOfFruitPerTruss,fruitingNode,imagePath
0,1,14,14,V001_tom1_00_061_e_14_20220101_27_05094434_401...
1,1,15,15,V001_tom1_00_061_e_15_20220101_27_06094401_401...
2,1,14,14,V001_tom1_00_062_e_14_20220101_27_05094518_401...
3,1,15,15,V001_tom1_00_062_e_15_20220101_27_05094550_401...
4,1,15,15,V001_tom1_00_062_e_15_20220101_27_06094550_401...
...,...,...,...,...
27295,1,06,06,V001_tom6_49_031_e_06_20211209_14_06104715_376...
27296,1,07,07,V001_tom6_49_031_e_07_20211209_14_08104831_376...
27297,1,03,03,V001_tom6_49_032_e_03_20211209_14_04155115_376...
27298,1,05,05,V001_tom6_49_032_e_05_20211209_14_04150227_376...


In [121]:
# f 만개꽃

In [122]:
tomato_dir = "C:/juna27/PythonWork/AI/data/tomato_label/지능형 스마트팜 통합데이터(토마토)_json/TL_f만개꽃/f.만개꽃"

tomato_json_data = []
for file_name in tqdm(os.listdir(tomato_dir)):
    if file_name.endswith(".json"):
        file_path = os.path.join(tomato_dir, file_name)
        with open(file_path, "r", encoding="utf-8") as target_file:
            tomato_json_data.append(json.load(target_file))

100%|██████████████████████████████████████████████████████████████████████████████| 1470/1470 [00:18<00:00, 78.72it/s]


In [123]:
f1_data = tomato_json_data

In [124]:
tomato_f1_df = mk_tomato_df(f1_data)
tomato_f1_df.isnull().sum()

100%|█████████████████████████████████████████████████████████████████████████████| 1470/1470 [00:01<00:00, 881.77it/s]


imagePath        0
imageHeight      0
imageWidth       0
fileVersion      0
farmId           0
week             0
number           0
type             0
flowerCluster    0
date             0
dtype: int64

In [125]:
tomato_f1_df

,imagePath,imageHeight,imageWidth,fileVersion,farmId,week,number,type,flowerCluster,date
0,V001_tom1_39_001_f_09_20210930_14_01135300_491...,1920,1080,V001,tom1,39,001,f,09,20210930
1,V001_tom1_39_002_f_08_20210930_14_00134624_491...,1920,1080,V001,tom1,39,002,f,08,20210930
2,V001_tom1_39_003_f_08_20210930_14_00133352_491...,1920,1080,V001,tom1,39,003,f,08,20210930
3,V001_tom1_39_006_f_09_20210930_14_00131114_491...,1920,1080,V001,tom1,39,006,f,09,20210930
4,V001_tom1_39_008_f_09_20210930_14_00125415_491...,1920,1080,V001,tom1,39,008,f,09,20210930
...,...,...,...,...,...,...,...,...,...,...
1465,V001_tom6_48_026_f_09_20211202_13_00100240_376...,1920,1080,V001,tom6,48,026,f,09,20211202
1466,V001_tom6_48_026_f_09_20211202_13_01100240_376...,1920,1080,V001,tom6,48,026,f,09,20211202
1467,V001_tom6_48_027_f_07_20211202_13_00144857_376...,1920,1080,V001,tom6,48,027,f,07,20211202
1468,V001_tom6_48_027_f_07_20211202_13_01144857_376...,1920,1080,V001,tom6,48,027,f,07,20211202


In [126]:
growth_f1_df = mk_growth_df(f1_data)
growth_f1_df.isnull().sum()

100%|█████████████████████████████████████████████████████████████████████████| 1470/1470 [00:00<00:00, 1436874.13it/s]


floweringNode                      0
numberOfTheFlower                  0
numberOfTheFlowerPerTruss          0
numberOfTheFlowerFullyDeveloped    2
imagePath                          0
dtype: int64

In [127]:
growth_f1_df

,floweringNode,numberOfTheFlower,numberOfTheFlowerPerTruss,numberOfTheFlowerFullyDeveloped,imagePath
0,09,1,09,1.0,V001_tom1_39_001_f_09_20210930_14_01135300_491...
1,08,1,08,1.0,V001_tom1_39_002_f_08_20210930_14_00134624_491...
2,08,1,08,1.0,V001_tom1_39_003_f_08_20210930_14_00133352_491...
3,09,1,09,1.0,V001_tom1_39_006_f_09_20210930_14_00131114_491...
4,09,1,09,1.0,V001_tom1_39_008_f_09_20210930_14_00125415_491...
...,...,...,...,...,...
1465,09,1,09,1.0,V001_tom6_48_026_f_09_20211202_13_00100240_376...
1466,09,1,09,1.0,V001_tom6_48_026_f_09_20211202_13_01100240_376...
1467,07,1,07,1.0,V001_tom6_48_027_f_07_20211202_13_00144857_376...
1468,07,1,07,1.0,V001_tom6_48_027_f_07_20211202_13_01144857_376...


### df 확인

In [148]:
# 선언한 변수명 모아두기
to_df_names = ["tomato_a1_df",
"tomato_a2_df",
"tomato_a3_df",
"tomato_a4_df",
"tomato_b1_df",
"tomato_c1_df",
"tomato_d1_df",
"tomato_d2_df",
"tomato_e1_df",
"tomato_f1_df"]

In [158]:
# 코드 반복 작성을 위한 코드
to_df_code = []
for i in range(len(to_df_names)):
    to_df_code.append(f'{to_df_names[i]}.to_csv("data/tomato_label/tomato_df/{to_df_names[i]}.csv", encoding="utf-8-sig", index=False)')

In [159]:
to_df_code

['tomato_a1_df.to_csv("data/tomato_label/tomato_df/tomato_a1_df.csv", encoding="utf-8-sig", index=False)',
 'tomato_a2_df.to_csv("data/tomato_label/tomato_df/tomato_a2_df.csv", encoding="utf-8-sig", index=False)',
 'tomato_a3_df.to_csv("data/tomato_label/tomato_df/tomato_a3_df.csv", encoding="utf-8-sig", index=False)',
 'tomato_a4_df.to_csv("data/tomato_label/tomato_df/tomato_a4_df.csv", encoding="utf-8-sig", index=False)',
 'tomato_b1_df.to_csv("data/tomato_label/tomato_df/tomato_b1_df.csv", encoding="utf-8-sig", index=False)',
 'tomato_c1_df.to_csv("data/tomato_label/tomato_df/tomato_c1_df.csv", encoding="utf-8-sig", index=False)',
 'tomato_d1_df.to_csv("data/tomato_label/tomato_df/tomato_d1_df.csv", encoding="utf-8-sig", index=False)',
 'tomato_d2_df.to_csv("data/tomato_label/tomato_df/tomato_d2_df.csv", encoding="utf-8-sig", index=False)',
 'tomato_e1_df.to_csv("data/tomato_label/tomato_df/tomato_e1_df.csv", encoding="utf-8-sig", index=False)',
 'tomato_f1_df.to_csv("data/tomato_la

In [172]:
# tomato_a1_df.to_csv("data/tomato_label/tomato_df/tomato_a1_df.csv", encoding="utf-8-sig", index=False)
# tomato_a2_df.to_csv("data/tomato_label/tomato_df/tomato_a2_df.csv", encoding="utf-8-sig", index=False)
# tomato_a3_df.to_csv("data/tomato_label/tomato_df/tomato_a3_df.csv", encoding="utf-8-sig", index=False)
# tomato_a4_df.to_csv("data/tomato_label/tomato_df/tomato_a4_df.csv", encoding="utf-8-sig", index=False)
# tomato_b1_df.to_csv("data/tomato_label/tomato_df/tomato_b1_df.csv", encoding="utf-8-sig", index=False)
# tomato_c1_df.to_csv("data/tomato_label/tomato_df/tomato_c1_df.csv", encoding="utf-8-sig", index=False)
# tomato_d1_df.to_csv("data/tomato_label/tomato_df/tomato_d1_df.csv", encoding="utf-8-sig", index=False)
# tomato_d2_df.to_csv("data/tomato_label/tomato_df/tomato_d2_df.csv", encoding="utf-8-sig", index=False)
# tomato_e1_df.to_csv("data/tomato_label/tomato_df/tomato_e1_df.csv", encoding="utf-8-sig", index=False)
# tomato_f1_df.to_csv("data/tomato_label/tomato_df/tomato_f1_df.csv", encoding="utf-8-sig", index=False)

In [162]:
# growth_a1_df.to_csv("data/tomato_label/growth_df/growth_a1_df.csv", encoding="utf-8-sig", index=False)
# growth_a2_df.to_csv("data/tomato_label/growth_df/growth_a2_df.csv", encoding="utf-8-sig", index=False)
# growth_a3_df.to_csv("data/tomato_label/growth_df/growth_a3_df.csv", encoding="utf-8-sig", index=False)
# growth_a4_df.to_csv("data/tomato_label/growth_df/growth_a4_df.csv", encoding="utf-8-sig", index=False)
# growth_b1_df.to_csv("data/tomato_label/growth_df/growth_b1_df.csv", encoding="utf-8-sig", index=False)
# growth_c1_df.to_csv("data/tomato_label/growth_df/growth_c1_df.csv", encoding="utf-8-sig", index=False)
# growth_d1_df.to_csv("data/tomato_label/growth_df/growth_d1_df.csv", encoding="utf-8-sig", index=False)
# growth_d2_df.to_csv("data/tomato_label/growth_df/growth_d2_df.csv", encoding="utf-8-sig", index=False)
# growth_e1_df.to_csv("data/tomato_label/growth_df/growth_e1_df.csv", encoding="utf-8-sig", index=False)
# growth_f1_df.to_csv("data/tomato_label/growth_df/growth_f1_df.csv", encoding="utf-8-sig", index=False)

In [167]:
# 파일 불러와서 확인
pd.read_csv("data/tomato_label/tomato_df/tomato_a1_df.csv")

,imagePath,imageHeight,imageWidth,fileVersion,farmId,week,number,type,flowerCluster,date
0,V001_tom1_39_001_a1_00_20210930_14_00134902_49...,1920,1080,V001,tom1,39,1,a1,0,20210930
1,V001_tom1_39_006_a1_00_20210930_14_00130645_49...,1920,1080,V001,tom1,39,6,a1,0,20210930
2,V001_tom1_39_010_a1_00_20210930_14_00124109_49...,1920,1080,V001,tom1,39,10,a1,0,20210930
3,V001_tom1_39_011_a1_00_20210930_14_00122602_49...,1920,1080,V001,tom1,39,11,a1,0,20210930
4,V001_tom1_39_011_a1_00_20210930_14_00124155_49...,1920,1080,V001,tom1,39,11,a1,0,20210930
...,...,...,...,...,...,...,...,...,...,...
17495,V001_tom6_47_097_a1_00_20211124_12_03094115_76...,1920,1080,V001,tom6,47,97,a1,0,20211124
17496,V001_tom6_47_097_a1_00_20211124_12_04094115_76...,1920,1080,V001,tom6,47,97,a1,0,20211124
17497,V001_tom6_47_098_a1_00_20211124_12_01095035_76...,1920,1080,V001,tom6,47,98,a1,0,20211124
17498,V001_tom6_47_098_a1_00_20211124_12_02095035_76...,1920,1080,V001,tom6,47,98,a1,0,20211124


In [169]:
# 기본 정보 통합
total_df = pd.concat([tomato_a1_df,
tomato_a2_df,
tomato_a3_df,
tomato_a4_df,
tomato_b1_df,
tomato_c1_df,
tomato_d1_df,
tomato_d2_df,
tomato_e1_df,
tomato_f1_df], ignore_index=True)
# print(total_df)

In [173]:
# total_df.to_csv("data/tomato_label/tomato_df/tomato_total_df.csv", encoding="utf-8-sig", index=False)

In [174]:
# 성장 정보 통합
total_growth_df = pd.concat([growth_a1_df,
growth_a2_df,
growth_a3_df,
growth_a4_df,
growth_b1_df,
growth_c1_df,
growth_d1_df,
growth_d2_df,
growth_e1_df,
growth_f1_df], ignore_index=True)

# print(total_df)

In [175]:
# 전체 데이터 통합

# total_growth_df.to_csv("data/tomato_label/growth_df/growth_total_df.csv", encoding="utf-8-sig", index=False)

In [176]:
# 확인
tomato_final_df = pd.merge(total_df, total_growth_df, on="imagePath", how="outer")
tomato_final_df

,imagePath,imageHeight,imageWidth,fileVersion,farmId,week,number,type,flowerCluster,date,...,numberOfLeaves,leafLength,leafWidth,floweringNode,numberOfTheFlower,numberOfTheFlowerPerTruss,numberOfFruitPerPlant,numberOfFruitPerTruss,fruitingNode,numberOfTheFlowerFullyDeveloped
0,V001_tom1_39_001_a1_00_20210930_14_00134902_49...,1920,1080,V001,tom1,39,001,a1,00,20210930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,V001_tom1_39_006_a1_00_20210930_14_00130645_49...,1920,1080,V001,tom1,39,006,a1,00,20210930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,V001_tom1_39_010_a1_00_20210930_14_00124109_49...,1920,1080,V001,tom1,39,010,a1,00,20210930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,V001_tom1_39_011_a1_00_20210930_14_00122602_49...,1920,1080,V001,tom1,39,011,a1,00,20210930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,V001_tom1_39_011_a1_00_20210930_14_00124155_49...,1920,1080,V001,tom1,39,011,a1,00,20210930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146365,V001_tom6_48_026_f_09_20211202_13_00100240_376...,1920,1080,V001,tom6,48,026,f,09,20211202,...,NaN,NaN,NaN,09,1.0,09,NaN,NaN,NaN,1.0
146366,V001_tom6_48_026_f_09_20211202_13_01100240_376...,1920,1080,V001,tom6,48,026,f,09,20211202,...,NaN,NaN,NaN,09,1.0,09,NaN,NaN,NaN,1.0
146367,V001_tom6_48_027_f_07_20211202_13_00144857_376...,1920,1080,V001,tom6,48,027,f,07,20211202,...,NaN,NaN,NaN,07,1.0,07,NaN,NaN,NaN,1.0
146368,V001_tom6_48_027_f_07_20211202_13_01144857_376...,1920,1080,V001,tom6,48,027,f,07,20211202,...,NaN,NaN,NaN,07,1.0,07,NaN,NaN,NaN,1.0


In [177]:
# 저장
# tomato_final_df.to_csv("data/tomato_label/tomato_df/tomato_final_df.csv", encoding="utf-8-sig", index=False)

In [14]:
tomato_df_0 = pd.read_csv("data/tomato_label/tomato_df/tomato_final_df.csv")

t_type_list = list(pd.read_csv("data/tomato_label/tomato_df/tomato_final_df.csv").type.unique())
t_type_list

['a1', 'a2', 'a3', 'a4', 'b', 'c', 'd1', 'd2', 'e', 'f']

In [20]:
print("토마토 이미지 .json 파일 중 Growth_indicator 정보 조회!")

for i in range(len(t_type_list)):
    tdf = tomato_df_0[tomato_df_0["type"] == f"{t_type_list[i]}"]
    print("="*50)    
    print(f"토마토 Type : {t_type_list[i]}")
    print(f"{t_type_list[i]}의 row 수 :",len(tdf))
    print(f"\n{t_type_list[i]} NULL 수")
    print("-"*50)
    print(tdf.isnull().sum()[10:])
    print("="*50)

토마토 이미지 .json 파일 중 Growth_indicator 정보 조회!
토마토 Type : a1
a1의 row 수 : 17500

a1 NULL 수
--------------------------------------------------
plantHeight                         3333
weeklyGrowth                        3333
heightOfTheFlowerTruss             17500
stemDiameter                       17500
numberOfLeaves                     17500
leafLength                         17500
leafWidth                          17500
floweringNode                      17500
numberOfTheFlower                  17500
numberOfTheFlowerPerTruss          17500
numberOfFruitPerPlant              17500
numberOfFruitPerTruss              17500
fruitingNode                       17500
numberOfTheFlowerFullyDeveloped    17500
dtype: int64
토마토 Type : a2
a2의 row 수 : 3500

a2 NULL 수
--------------------------------------------------
plantHeight                        3500
weeklyGrowth                       3500
heightOfTheFlowerTruss              328
stemDiameter                       3500
numberOfLeaves         

------------------------

### 2. 토마토 환경 정보 시계열 데이터  CSV -> DataFrame

tom 1, 4, 5, 6 csv파일 불러와서 데이터프레임으로 변환

In [5]:
#### tom 1, 4, 5, 6 file => df

# def mk_df(path):
    
name = ["00_(환경제어기, 양액기)", "01_(함수율저울,열화상센서,작물환경센서)"]
# file_path = "D:/097.지능형 스마트팜 통합 데이터(토마토)/01.데이터/1.Training/원천데이터/TS_Timeseries"
file_path = "C:/juna27/PythonWork/AI/data/tomato_label/smart/지능형 스마트팜 통합데이터(토마토)_JSON_TS_Timeseries"
file_list = []
env_list = []
dfs = []
to_file = os.path.join(file_path, "tom4")

##
for env in name:
    to_file_env = os.path.join(to_file, env)
    env_list.append(to_file_env)

for file in env_list:
    file_name = glob(file+"/*csv")
    
    for names in file_name:
        file_list.append(names)
    
for i in range(len(file_list)):
    
    try:
        df = pd.read_csv(file_list[i], encoding="cp949")
        
    except UnicodeDecodeError as e :
        
        df = pd.read_csv(file_list[i], encoding="utf-8")
        
    dfs.append(df) 

C:\Users\acorn\AppData\Local\Temp\ipykernel_32348\1190836429.py:27: DtypeWarning: Columns (3,4,15,16,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_list[i], encoding="cp949")


In [181]:
len(dfs)

5

In [6]:
tom4_dfs = dfs

In [184]:
print(len(tom4_dfs[0].columns))
print(len(tom4_dfs[1].columns))
print(len(tom4_dfs[2].columns))
print(len(tom4_dfs[3].columns))

print(len(tom4_dfs[4].columns))

68
68
68
68
16


In [185]:
# tom4

In [7]:
pd.set_option("display.max_columns", None)

In [8]:
tom4_1 = pd.DataFrame(tom4_dfs[0])
tom4_2 = pd.DataFrame(tom4_dfs[1])
tom4_3 = pd.DataFrame(tom4_dfs[2])
tom4_4 = pd.DataFrame(tom4_dfs[3])

In [9]:
tom4_00 = pd.concat([tom4_1, tom4_2, tom4_3, tom4_4], ignore_index=True)
tom4_00

,채널1 구역,날짜,시간,설정량1,설정량2,설정량3,설정량4,설정량5,설정량6,설정량7,설정량8,설정량9,설정량10,설정량11,설정량12,잔여량1,잔여량2,잔여량3,잔여량4,잔여량5,잔여량6,잔여량7,잔여량8,잔여량9,잔여량10,잔여량11,잔여량12,누적량1,누적량2,누적량3,누적량4,누적량5,누적량6,누적량7,누적량8,누적량9,누적량10,누적량11,누적량12,시작,종료,설정EC(dS),현재EC(dS),설정PH(pH),현재PH(pH),일출시간,일몰시간,현재일사(W),누적일사(J),유량펄스,함수율(%),배지EC(dS),배지온도(도),실내온도(도),실내습도(도),수분부족분(g/M),배관EC(dS),G1,G2,G3,J/30min,J/Day,경보내용,경보데이터,동작구역회수,전구역관수횟수,유량센서카운트,유량(L)
0,0,2021-10-1,0:0:34,3:40,3:20,,,,,,,,,,,3:40,3:20,,,,,,,,,,,0:0,0:0,,,,,,,,,,,6:58,15:16,2.8,2.8,5.6,5.6,6:28,18:16,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1330,경보없음,0:0:0:0:0:0:0:0:0:0:,0,0:0:0:0:0:0:0:0:0:0:0:0:,0,0
1,0,2021-10-1,0:1:37,3:40,3:20,,,,,,,,,,,3:40,3:20,,,,,,,,,,,0:0,0:0,,,,,,,,,,,6:58,15:16,2.8,2.7,5.6,5.6,6:28,18:16,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1330,경보없음,0:0:0:0:0:0:0:0:0:0:,0,0:0:0:0:0:0:0:0:0:0:0:0:,0,0
2,0,2021-10-1,0:2:30,3:40,3:20,,,,,,,,,,,3:40,3:20,,,,,,,,,,,0:0,0:0,,,,,,,,,,,6:58,15:16,2.8,2.8,5.6,5.6,6:28,18:16,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1330,경보없음,0:0:0:0:0:0:0:0:0:0:,0,0:0:0:0:0:0:0:0:0:0:0:0:,0,0
3,0,2021-10-1,0:3:26,3:40,3:20,,,,,,,,,,,3:40,3:20,,,,,,,,,,,0:0,0:0,,,,,,,,,,,6:58,15:16,2.8,2.8,5.6,5.7,6:28,18:16,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1330,경보없음,0:0:0:0:0:0:0:0:0:0:,0,0:0:0:0:0:0:0:0:0:0:0:0:,0,0
4,0,2021-10-1,0:4:29,3:40,3:20,,,,,,,,,,,3:40,3:20,,,,,,,,,,,0:0,0:0,,,,,,,,,,,6:58,15:16,2.8,2.8,5.6,5.6,6:28,18:16,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1330,경보없음,0:0:0:0:0:0:0:0:0:0:,0,0:0:0:0:0:0:0:0:0:0:0:0:,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166880,0,2022-1-25,14:25:49,48,48,,,,,,,,,,,0,0,,,,,,,,,,,371,371,,,,,,,,,,,8:12,18:00,2.0,2.0,5.9,4.3,7:42,17:47,122,355,0,0,0.0,0.0,0.0,0.0,0.0,0.0,130,0,0,0,255,경보없음,0:0:0:0:0:0:0:0:0:0:,0,7:7:0:0:0:0:0:0:0:0:0:0:,0,0
166881,0,2022-1-25,14:26:56,48,48,,,,,,,,,,,0,0,,,,,,,,,,,371,371,,,,,,,,,,,8:12,18:00,2.0,1.9,5.9,4.2,7:42,17:47,125,356,0,0,0.0,0.0,0.0,0.0,0.0,0.0,131,0,0,0,255,경보없음,0:0:0:0:0:0:0:0:0:0:,0,7:7:0:0:0:0:0:0:0:0:0:0:,0,0
166882,0,2022-1-25,14:27:48,48,48,,,,,,,,,,,0,0,,,,,,,,,,,371,371,,,,,,,,,,,8:12,18:00,2.0,1.9,5.9,4.2,7:42,17:47,137,357,0,0,0.0,0.0,0.0,0.0,0.0,0.0,132,0,0,0,255,경보없음,0:0:0:0:0:0:0:0:0:0:,0,7:7:0:0:0:0:0:0:0:0:0:0:,0,0
166883,0,2022-1-25,14:28:52,48,48,,,,,,,,,,,0,0,,,,,,,,,,,371,371,,,,,,,,,,,8:12,18:00,2.0,2.0,5.9,4.3,7:42,17:47,140,358,0,0,0.0,0.0,0.0,0.0,0.0,0.0,133,0,0,0,255,경보없음,0:0:0:0:0:0:0:0:0:0:,0,7:7:0:0:0:0:0:0:0:0:0:0:,0,0


In [117]:
tom4_env_df = tom4_dfs[4].copy()

In [ ]:
for i in range(len(tom4_env_df)):
    tom4_env_df["time"][i] = pd.to_datetime(tom4_time)[i].date()

In [37]:
tom4_env_df.head(3)

,time,EC급액,EC배액,pH급액,pH배액,저울급액,저울배액,온도토양,온도급액,온도배액,내부이산화탄소,내부광량,내부PT100온도센서1번건구,내부PT100온도센서2번습구,내부PT100온도센서3번,내부PT100센서를이용한계산습도
0,2021-11-10 18:28:37,80.27300,0.0,9.998,10.010,9.376138,0.000092,16.0,16.3125,16.375,391,0.0,16.40922,15.76818,16.10563,95.99306
1,2021-11-10 18:28:49,80.29279,0.0,9.999,10.007,9.373927,0.000341,16.0,16.3125,16.375,392,0.0,16.40922,15.76818,16.13930,95.99306
2,2021-11-10 18:29:00,80.30475,0.0,10.000,10.010,9.375060,0.000929,16.0,16.3125,16.375,391,0.0,16.37535,15.80184,16.10563,96.40750


In [38]:
len(tom4_env_cols)

16

In [108]:
pd.to_datetime(tom4_env_df["time"])

0      2021-11-10 18:28:37
1      2021-11-10 18:28:49
2      2021-11-10 18:29:00
3      2021-11-10 18:30:09
4      2021-11-10 18:30:21
               ...        
4750   2021-11-12 13:04:26
4751   2021-11-12 13:04:37
4752   2021-11-12 13:04:59
4753   2021-11-12 13:05:33
4754   2021-11-12 13:05:56
Name: time, Length: 4755, dtype: datetime64[ns]

In [104]:
tom4_env_df.time

0       2021-11-10 18:28:37
1       2021-11-10 18:28:49
2       2021-11-10 18:29:00
3       2021-11-10 18:30:09
4       2021-11-10 18:30:21
               ...         
4750    2021-11-12 13:04:26
4751    2021-11-12 13:04:37
4752    2021-11-12 13:04:59
4753    2021-11-12 13:05:33
4754    2021-11-12 13:05:56
Name: time, Length: 4755, dtype: object

In [172]:
pd.to_datetime(tom4_time)[0].date()

datetime.date(2021, 11, 10)

In [198]:
tom4_env_df

,time,EC급액,EC배액,pH급액,pH배액,저울급액,저울배액,온도토양,온도급액,온도배액,내부이산화탄소,내부광량,내부PT100온도센서1번건구,내부PT100온도센서2번습구,내부PT100온도센서3번,내부PT100센서를이용한계산습도
0,2021-11-10,80.27300,0.00000,9.998,10.010,9.376138,0.000092,16.0000,16.3125,16.3750,391,0.0000,16.40922,15.76818,16.10563,95.99306
1,2021-11-10,80.29279,0.00000,9.999,10.007,9.373927,0.000341,16.0000,16.3125,16.3750,392,0.0000,16.40922,15.76818,16.13930,95.99306
2,2021-11-10,80.30475,0.00000,10.000,10.010,9.375060,0.000929,16.0000,16.3125,16.3750,391,0.0000,16.37535,15.80184,16.10563,96.40750
3,2021-11-10,80.44090,0.00000,9.997,10.011,9.372601,0.000263,16.0000,16.3125,16.3750,389,0.0000,16.37535,15.70065,16.03810,95.78552
4,2021-11-10,80.41465,0.00000,9.995,9.990,9.370611,0.000263,15.9375,16.3125,16.3750,390,0.0000,16.37535,15.73451,16.03810,95.99328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4750,2021-11-12,2.61999,2.26627,6.827,7.031,5.855098,0.064705,18.7500,18.3750,18.6875,370,180.6928,22.01449,18.63673,26.68179,81.17117
4751,2021-11-12,2.62078,2.26876,6.827,7.032,5.852030,0.064705,18.7500,18.3750,18.6875,374,174.4005,22.04836,18.56920,26.64792,80.66219
4752,2021-11-12,2.62073,2.26966,6.825,7.031,5.851297,0.064072,18.8125,18.4375,18.6875,375,181.9921,22.08222,18.53554,26.44473,80.32641
4753,2021-11-12,2.62154,2.26818,6.829,7.029,5.847198,0.064382,18.8125,18.4375,18.6875,377,194.4492,22.11589,18.40048,26.41106,79.48650


In [197]:
for i in range(len(tom4_env_df)):
    tom4_env_df["time"][i] = pd.to_datetime(tom4_time)[i].date()

C:\Users\acorn\AppData\Local\Temp\ipykernel_34752\3475863194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tom4_env_df["time"][i] = pd.to_datetime(tom4_time)[i].date()
C:\Users\acorn\AppData\Local\Temp\ipykernel_34752\3475863194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tom4_env_df["time"][i] = pd.to_datetime(tom4_time)[i].date()
C:\Users\acorn\AppData\Local\Temp\ipykernel_34752\3475863194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

C:\Users\acorn\AppData\Local\Temp\ipykernel_34752\3475863194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tom4_env_df["time"][i] = pd.to_datetime(tom4_time)[i].date()
C:\Users\acorn\AppData\Local\Temp\ipykernel_34752\3475863194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tom4_env_df["time"][i] = pd.to_datetime(tom4_time)[i].date()
C:\Users\acorn\AppData\Local\Temp\ipykernel_34752\3475863194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

C:\Users\acorn\AppData\Local\Temp\ipykernel_34752\3475863194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tom4_env_df["time"][i] = pd.to_datetime(tom4_time)[i].date()
C:\Users\acorn\AppData\Local\Temp\ipykernel_34752\3475863194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tom4_env_df["time"][i] = pd.to_datetime(tom4_time)[i].date()
C:\Users\acorn\AppData\Local\Temp\ipykernel_34752\3475863194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

C:\Users\acorn\AppData\Local\Temp\ipykernel_34752\3475863194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tom4_env_df["time"][i] = pd.to_datetime(tom4_time)[i].date()
C:\Users\acorn\AppData\Local\Temp\ipykernel_34752\3475863194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tom4_env_df["time"][i] = pd.to_datetime(tom4_time)[i].date()
C:\Users\acorn\AppData\Local\Temp\ipykernel_34752\3475863194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

C:\Users\acorn\AppData\Local\Temp\ipykernel_34752\3475863194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tom4_env_df["time"][i] = pd.to_datetime(tom4_time)[i].date()
C:\Users\acorn\AppData\Local\Temp\ipykernel_34752\3475863194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tom4_env_df["time"][i] = pd.to_datetime(tom4_time)[i].date()
C:\Users\acorn\AppData\Local\Temp\ipykernel_34752\3475863194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

...

C:\Users\acorn\AppData\Local\Temp\ipykernel_34752\3475863194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tom4_env_df["time"][i] = pd.to_datetime(tom4_time)[i].date()
C:\Users\acorn\AppData\Local\Temp\ipykernel_34752\3475863194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tom4_env_df["time"][i] = pd.to_datetime(tom4_time)[i].date()
C:\Users\acorn\AppData\Local\Temp\ipykernel_34752\3475863194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

C:\Users\acorn\AppData\Local\Temp\ipykernel_34752\3475863194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tom4_env_df["time"][i] = pd.to_datetime(tom4_time)[i].date()
C:\Users\acorn\AppData\Local\Temp\ipykernel_34752\3475863194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tom4_env_df["time"][i] = pd.to_datetime(tom4_time)[i].date()
C:\Users\acorn\AppData\Local\Temp\ipykernel_34752\3475863194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

C:\Users\acorn\AppData\Local\Temp\ipykernel_34752\3475863194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tom4_env_df["time"][i] = pd.to_datetime(tom4_time)[i].date()
C:\Users\acorn\AppData\Local\Temp\ipykernel_34752\3475863194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tom4_env_df["time"][i] = pd.to_datetime(tom4_time)[i].date()
C:\Users\acorn\AppData\Local\Temp\ipykernel_34752\3475863194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

C:\Users\acorn\AppData\Local\Temp\ipykernel_34752\3475863194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tom4_env_df["time"][i] = pd.to_datetime(tom4_time)[i].date()
C:\Users\acorn\AppData\Local\Temp\ipykernel_34752\3475863194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tom4_env_df["time"][i] = pd.to_datetime(tom4_time)[i].date()
C:\Users\acorn\AppData\Local\Temp\ipykernel_34752\3475863194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

C:\Users\acorn\AppData\Local\Temp\ipykernel_34752\3475863194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tom4_env_df["time"][i] = pd.to_datetime(tom4_time)[i].date()
C:\Users\acorn\AppData\Local\Temp\ipykernel_34752\3475863194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tom4_env_df["time"][i] = pd.to_datetime(tom4_time)[i].date()
C:\Users\acorn\AppData\Local\Temp\ipykernel_34752\3475863194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

In [210]:
tom4_env_df.head()

,time,EC급액,EC배액,pH급액,pH배액,저울급액,저울배액,온도토양,온도급액,온도배액,내부이산화탄소,내부광량,내부PT100온도센서1번건구,내부PT100온도센서2번습구,내부PT100온도센서3번,내부PT100센서를이용한계산습도
0,2021-11-10,80.27300,0.0,9.998,10.010,9.376138,0.000092,16.0000,16.3125,16.375,391,0.0,16.40922,15.76818,16.10563,95.99306
1,2021-11-10,80.29279,0.0,9.999,10.007,9.373927,0.000341,16.0000,16.3125,16.375,392,0.0,16.40922,15.76818,16.13930,95.99306
2,2021-11-10,80.30475,0.0,10.000,10.010,9.375060,0.000929,16.0000,16.3125,16.375,391,0.0,16.37535,15.80184,16.10563,96.40750
3,2021-11-10,80.44090,0.0,9.997,10.011,9.372601,0.000263,16.0000,16.3125,16.375,389,0.0,16.37535,15.70065,16.03810,95.78552
4,2021-11-10,80.41465,0.0,9.995,9.990,9.370611,0.000263,15.9375,16.3125,16.375,390,0.0,16.37535,15.73451,16.03810,95.99328


In [211]:
tom4_env_df.describe()

,EC급액,EC배액,pH급액,pH배액,저울급액,저울배액,온도토양,온도급액,온도배액,내부이산화탄소,내부광량,내부PT100온도센서1번건구,내부PT100온도센서2번습구,내부PT100온도센서3번,내부PT100센서를이용한계산습도
count,4755.000000,4755.000000,4755.000000,4755.000000,4755.000000,4755.000000,4755.000000,4755.000000,4755.000000,4755.000000,4755.000000,4755.000000,4755.000000,4755.000000,4755.000000
mean,2.525508,2.253399,6.542298,6.723893,5.302475,-14.836389,14.614550,14.864261,15.041404,508.033859,25.196639,14.953658,13.523221,15.420599,91.437322
std,2.528667,0.233021,0.243957,0.298410,0.665880,121.303984,2.220412,1.954253,2.037001,75.455046,55.941270,2.413152,1.826203,3.543471,6.127213
min,0.006890,0.000000,6.129000,6.323000,0.001658,-1000.000000,12.000000,12.750000,12.812500,280.000000,0.000000,12.564990,11.385720,12.261810,72.985210
25%,2.369515,2.104455,6.356000,6.529000,5.086337,0.077421,12.875000,13.375000,13.500000,443.000000,0.000000,13.205430,12.228140,13.036900,92.371695
50%,2.421480,2.258510,6.526000,6.699000,5.149985,0.081384,13.937500,14.187500,14.312500,522.000000,0.000000,13.778530,12.902040,13.576140,94.034580
75%,2.504465,2.374860,6.720000,6.939000,5.281083,0.119877,15.750000,15.812500,15.875000,568.000000,0.254748,16.071770,14.486700,16.814000,94.862030
max,80.440900,2.765990,10.000000,10.011000,20.324700,9.822846,20.937500,24.500000,27.437500,635.000000,366.837300,23.975310,22.217290,28.882290,97.009030


In [193]:
tom4_env = tom4_env_df.groupby("time").agg({"EC급액": "mean", "EC배액": "mean", "pH급액": "mean", "pH배액": "mean", 
                                            "저울급액": "mean", "저울배액": "mean", "온도토양": "mean", "온도급액": "mean", 
                                            "온도배액": "mean", "내부이산화탄소": "mean", "내부광량": "mean",
                                            "내부PT100온도센서1번건구": "mean", 
                                            "내부PT100온도센서2번습구": "mean",
                                            "내부PT100온도센서3번": "mean",
                                            "내부PT100센서를이용한계산습도": "mean"}).reset_index()


print(tom4_env)

         time      EC급액      EC배액      pH급액      pH배액      저울급액        저울배액  \
0  2021-11-10  3.067891  2.416841  6.321191  6.474630  5.858058 -104.446802   
1  2021-11-11  2.470255  2.314131  6.462610  6.629770  5.223456    0.101825   
2  2021-11-12  2.383389  2.096450  6.750976  6.965469  5.189964    0.075906   

        온도토양       온도급액       온도배액     내부이산화탄소       내부광량  내부PT100온도센서1번건구  \
0  16.072809  15.996870  16.021907  442.303387   0.006791        14.887885   
1  14.952115  15.170749  15.384649  499.648827  24.941905        15.194086   
2  13.515521  13.945317  14.130958  547.498482  35.957208        14.626190   

   내부PT100온도센서2번습구  내부PT100온도센서3번  내부PT100센서를이용한계산습도  
0        13.988647      14.625021          94.354635  
1        13.785095      15.633681          91.573478  
2        12.945128      15.434334          90.033814  


In [194]:
# time을 1day 기준으로 정리 , 농장 tom4
tom4_env

,time,EC급액,EC배액,pH급액,pH배액,저울급액,저울배액,온도토양,온도급액,온도배액,내부이산화탄소,내부광량,내부PT100온도센서1번건구,내부PT100온도센서2번습구,내부PT100온도센서3번,내부PT100센서를이용한계산습도
0,2021-11-10,3.067891,2.416841,6.321191,6.474630,5.858058,-104.446802,16.072809,15.996870,16.021907,442.303387,0.006791,14.887885,13.988647,14.625021,94.354635
1,2021-11-11,2.470255,2.314131,6.462610,6.629770,5.223456,0.101825,14.952115,15.170749,15.384649,499.648827,24.941905,15.194086,13.785095,15.633681,91.573478
2,2021-11-12,2.383389,2.096450,6.750976,6.965469,5.189964,0.075906,13.515521,13.945317,14.130958,547.498482,35.957208,14.626190,12.945128,15.434334,90.033814
